In [78]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
import pandas as pd 


from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4


def main():

    # Check command-line arguments
    if len(sys.argv) not in [2, 3]:
        sys.exit("Usage: python traffic.py data_directory [model.h5]")

    # Get image arrays and labels for all image files
    images, labels = load_data(sys.argv[1])

    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )

    # Get a compiled neural network
    model = get_model()

    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)
    print(classification_report(np.argmax(y_test, axis=1), y_pred))
    # Save model to file
    if len(sys.argv) == 3:
        filename = sys.argv[2]
        model.save(filename)
        print(f"Model saved to {filename}.")


def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    #first set required dimension
    required_dimension = (IMG_WIDTH,IMG_HEIGHT)

    #now loop through every input file
    labels = []
    images = []
    for root, subdirectories, files in os.walk(data_dir):
        for file in files:
            path = os.path.join(root, file)
            split_path = path.split(os.sep)
            # if subdirectory not an integer, just skip
            label = split_path[-2]
            try:
                int(label)
            except:
                continue
            # get image into array format
            image = cv2.imread(path)
            image = cv2.resize(image,required_dimension)
            image = image / 255.0
            labels.append(label)
            images.append(image)

    return (images,labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """

    # usually I use a notebook to record all my attempts, but since I do this too often at my job, I will not write a beautiful report as practise
    
    model = tf.keras.models.Sequential([

    # Convolutional layer. Learn 32 filters using a 3x3 kernel
    tf.keras.layers.Conv2D(
        64, 3, activation="relu", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
    ),

    # Max-pooling layer, using 2x2 pool size
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(
        64, 3, activation="relu", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
    ),    
    #tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Flatten units
    tf.keras.layers.Flatten(),

    # Add a hidden layer with dropout
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.1),  

    #tf.keras.layers.Dense(1280, activation="relu"),
    #tf.keras.layers.Dropout(0.2),  


    # Add an output layer with output units for all 10 digits
    tf.keras.layers.Dense(43, activation="softmax")
    ])

    # Train neural network
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


#if __name__ == "__main__":
#    main()


In [81]:
images, labels = load_data("./gtsrb/")
print(len(labels))

# Split data into training and testing sets
labels = tf.keras.utils.to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE
)

# Get a compiled neural network
model = get_model()

# Fit model on training data
model.fit(x_train, y_train, epochs=EPOCHS)

# Evaluate neural network performance
model.evaluate(x_test,  y_test, verbose=2)

y_pred = model.predict_classes(x_test)
print(classification_report(np.argmax(y_test, axis=1), y_pred))

26640
Epoch 1/10
500/500 [==============================] - 12s 24ms/step - loss: 2.1604 - accuracy: 0.4354
Epoch 2/10
500/500 [==============================] - 12s 24ms/step - loss: 0.2955 - accuracy: 0.9164
Epoch 3/10
500/500 [==============================] - 12s 24ms/step - loss: 0.1362 - accuracy: 0.9625
Epoch 4/10
500/500 [==============================] - 12s 25ms/step - loss: 0.0964 - accuracy: 0.9697
Epoch 5/10
500/500 [==============================] - 14s 27ms/step - loss: 0.0635 - accuracy: 0.9811
Epoch 6/10
500/500 [==============================] - 12s 25ms/step - loss: 0.0513 - accuracy: 0.9833
Epoch 7/10
500/500 [==============================] - 12s 25ms/step - loss: 0.0458 - accuracy: 0.9863
Epoch 8/10
500/500 [==============================] - 12s 25ms/step - loss: 0.0383 - accuracy: 0.9877
Epoch 9/10
500/500 [==============================] - 13s 25ms/step - loss: 0.0438 - accuracy: 0.9874
Epoch 10/10
500/500 [==============================] - 12s 25ms/step - loss:

In [82]:


cfx_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred, labels = [x for x in range(0,43)])

pd.DataFrame(cfx_matrix).to_csv("cfx4.csv")
#cfx_matrix.tofile('cfx_matrix.csv',sep=',')

#np.savetxt("confusion_matrix.csv", a, delimiter=",")


In [71]:
confusion_matrix(np.argmax(y_test, axis=1), y_pred)

array([[ 56,   9,   0, ...,   0,   0,   0],
       [  1, 571,   6, ...,   0,   0,   0],
       [  0,  20, 568, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 101,   0,   0],
       [  0,   0,   0, ...,   0,  59,   2],
       [  0,   0,   0, ...,   0,   0,  84]])

In [68]:
y_pred

array([10, 12, 30, ..., 23, 15, 15])

In [ ]:
    labels = []
    images = []
    required_dimension = (IMG_WIDTH,IMG_HEIGHT)
    for root, subdirectories, files in os.walk("./gtsrb/"):
        print(files)
        for file in files:
            print(file)
            path = os.path.join(root, file)
            split_path = path.split(os.sep)
            # if subdirectory not an integer, just skip
            label = split_path[-2]
            try:
                int(label)
            except:
                continue
            # get image into array format
            image = cv2.imread(path)
            image = cv2.resize(image,required_dimension)
            image = image / 255.0
            labels.append(label)
            images.append(image)


In [ ]:
images

In [ ]:
os.walk("./gtsrb/")

In [ ]:
image_dirs[1]

In [ ]:
im = cv2.imread(path)


In [ ]:
len(images)

In [ ]:
len(labels)

In [ ]:
im[56]

In [ ]:
print(os.sep)